In [1]:
from google.colab import drive
import os
import cv2
import math
import numpy as np
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

drive.mount('/content/drive')



Mounted at /content/drive


In [4]:
# Defining Image Quality Metric Functions
def psnr(target, ref):
    target_data = target.astype(float)
    ref_data = ref.astype(float)
    diff = ref_data - target_data
    diff = diff.flatten('C')
    rmse = math.sqrt(np.mean(diff ** 2.))
    return 20 * math.log10(255. / rmse)

def mse(target, ref):
    err = np.sum((target.astype('float') - ref.astype('float')) ** 2)
    err /= float(target.shape[0] * target.shape[1])
    return err

def compare_images(target, ref):
    return [psnr(target, ref), mse(target, ref), ssim(target, ref, multichannel=True)]

# Defining the SRCNN Model
def model():
    SRCNN = Sequential([
        Conv2D(128, (9, 9), activation='relu', padding='same', input_shape=(None, None, 1), kernel_initializer='glorot_uniform'),
        Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='glorot_uniform'),
        Conv2D(1, (5, 5), activation='linear', padding='same', kernel_initializer='glorot_uniform')
    ])
    SRCNN.compile(optimizer=Adam(lr=0.0003), loss='mean_squared_error', metrics=['mean_squared_error'])
    return SRCNN

# Function to prepare images
def prepare_images(path, factor, save_dir):
    try:
        files = os.listdir(path)
        print(f"Found {len(files)} files in {path}")
    except OSError as e:
        print(f"Error reading directory {path}: {e}")
        return

    for file in files:
        try:
            img_path = os.path.join(path, file)
            img = cv2.imread(img_path)
            h, w, _ = img.shape
            img = cv2.resize(img, (w // factor, h // factor), interpolation=cv2.INTER_LINEAR)
            img = cv2.resize(img, (w, h), interpolation=cv2.INTER_LINEAR)
            cv2.imwrite(os.path.join(save_dir, file), img)
            print(f"Processed and saved: {file}")
        except Exception as e:
            print(f"Error processing file {file}: {e}")


In [5]:
# Prepare directories
source_dir = '/content/drive/MyDrive/aqua_data/invasive'
degraded_dir = '/content/drive/MyDrive/SRCNN/degrated'
output_dir = '/content/drive/MyDrive/SRCNN/output'

# Run the preparation function
prepare_images(source_dir, 2, degraded_dir)




Streaming output truncated to the last 5000 lines.
Processed and saved: I981 (8).png
Processed and saved: I287 (5).png
Processed and saved: I1686 (6).png
Processed and saved: I718 (18).png
Processed and saved: I1377 (22).png
Processed and saved: I1280 (18).png
Processed and saved: I2872 (21).png
Processed and saved: I2189 (12).png
Processed and saved: I645 (12).png
Processed and saved: I763 (3).png
Processed and saved: I3492 (21).png
Processed and saved: I643 (1).png
Processed and saved: I3751 (14).png
Processed and saved: I2698 (4).png
Processed and saved: I419 (19).png
Processed and saved: I1261 (15).png
Processed and saved: I3929 (34).png
Processed and saved: I1240 (1).png
Processed and saved: I3981 (29).png
Processed and saved: I537 (26).png
Processed and saved: I1388 (4).png
Processed and saved: I446 (27).png
Processed and saved: I833 (11).png
Processed and saved: I663 (19).png
Processed and saved: I2738 (29).png
Processed and saved: I1602 (22).png
Processed and saved: I1826 (10).

In [6]:
# Load the SRCNN model
srcnn = model()

# Load weights if you have them (optional)
srcnn.load_weights('/content/drive/MyDrive/SRCNN/3051crop_weight_200.h5')



In [ ]:
# Function to predict and enhance images
def predict_and_enhance(image_path, srcnn):
    path, file = os.path.split(image_path)
    degraded = cv2.imread(image_path)
    ref = cv2.imread(os.path.join(source_dir, file))

    # Convert to grayscale for SRCNN
    temp = cv2.cvtColor(degraded, cv2.COLOR_BGR2YCrCb)
    Y = np.zeros((1, temp.shape[0], temp.shape[1], 1), dtype=float)
    Y[0, :, :, 0] = temp[:, :, 0].astype(float) / 255

    # Predict with SRCNN
    pre = srcnn.predict(Y, batch_size=1)
    pre = pre * 255
    pre[pre > 255] = 255
    pre[pre < 0] = 0
    pre = pre.astype(np.uint8)

    # Restore color and save
    temp[:, :, 0] = pre[0, :, :, 0]
    output = cv2.cvtColor(temp, cv2.COLOR_YCrCb2BGR)
    cv2.imwrite(os.path.join(output_dir, file), output)

    # Calculate image quality scores
    scores = compare_images(output, ref)
    print(f'Processed {file} with PSNR: {scores[0]}, MSE: {scores[1]}, SSIM: {scores[2]}')

# Process each degraded image
for file in os.listdir(degraded_dir):
    try:
        predict_and_enhance(os.path.join(degraded_dir, file), srcnn)
    except Exception as e:
        print(f'Failed to process {file}: {e}')


1/1 [==============================] - 1s 503ms/step
Processed Im154 (7).png with PSNR: 33.798846254977384, MSE: 81.34240150093808, SSIM: 0.9626281614185442
1/1 [==============================] - 0s 99ms/step


<ipython-input-4-5a5e6521f5c1>:16: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  return [psnr(target, ref), mse(target, ref), ssim(target, ref, multichannel=True)]


Streaming output truncated to the last 5000 lines.
Processed I2872 (20).png with PSNR: 33.231083597385094, MSE: 92.70288624787776, SSIM: 0.9670723384855645
1/1 [==============================] - 0s 55ms/step
Processed I763 (6).png with PSNR: 29.47425824470554, MSE: 220.17894736842106, SSIM: 0.9394640431338152
1/1 [==============================] - 0s 37ms/step
Processed I2907 (32).png with PSNR: 27.86646627675148, MSE: 318.8269230769231, SSIM: 0.9294182371263279
1/1 [==============================] - 0s 57ms/step
Processed I4045 (16).png with PSNR: 25.257265881531637, MSE: 581.4, SSIM: 0.8846817438006299
1/1 [==============================] - 0s 47ms/step
Processed I2153 (7).png with PSNR: 33.01786003363679, MSE: 97.36785714285715, SSIM: 0.9430706951056198
1/1 [==============================] - 0s 90ms/step
Processed I733 (7).png with PSNR: 29.62928781294046, MSE: 212.45787545787545, SSIM: 0.9397870146641317
1/1 [==============================] - 0s 96ms/step
Processed I1752 (23).png w